In [0]:
x = 5
y = 5
print(x+y)

list = [1,2,3,4]
print(sum(list))

10
10


In [0]:
list = [1,2,3,4]
print(sum(list)/len(list))

2.5


In [0]:
# Tuple 
my_tuple = (10,20,70)
print(my_tuple)

(10, 20, 70)


In [0]:
my_dict = {"Name": "Naitik", "Age": 21}
print(my_dict["Name"])    # output Hardik
my_dict["Age"] = 22
print(my_dict["Age"])     # output 22
print(my_dict.keys())     # output name , age 
print(my_dict)

Naitik
22
dict_keys(['Name', 'Age'])
{'Name': 'Naitik', 'Age': 22}


In [0]:
my_set = {1,2,3,4,5}
my_set.add(6)
print(my_set)
my_set.update([2,7,8])  # add multiple values
print(my_set)

{1, 2, 3, 4, 5, 6}
{1, 2, 3, 4, 5, 6, 7, 8}


In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PythonDF").getOrCreate()

# Creating DataFrame from list of tuples
data = [(1, "Alice"), (2, "Bob"), (3, "Carol")]
df = spark.createDataFrame(data, ["id", "name"])
df.show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
|  3|Carol|
+---+-----+



In [0]:
# Select specific columns
df.select("name").show()

# Filter rows
df.filter(df["id"] > 1).show()

# Count rows
print(df.count())

# Describe summary statistics (numeric columns)
df.describe().show()

# Add new column with literal value
from pyspark.sql.functions import lit
df = df.withColumn("country", lit("India"))
df.show()

+-----+
| name|
+-----+
|Alice|
|  Bob|
|Carol|
+-----+

+---+-----+-------+
| id| name|country|
+---+-----+-------+
|  2|  Bob|  India|
|  3|Carol|  India|
+---+-----+-------+

3
+-------+---+-----+-------+
|summary| id| name|country|
+-------+---+-----+-------+
|  count|  3|    3|      3|
|   mean|2.0| null|   null|
| stddev|1.0| null|   null|
|    min|  1|Alice|  India|
|    max|  3|Carol|  India|
+-------+---+-----+-------+

+---+-----+-------+
| id| name|country|
+---+-----+-------+
|  1|Alice|  India|
|  2|  Bob|  India|
|  3|Carol|  India|
+---+-----+-------+



In [0]:
sales_data = [
    ("2024-01-01", "North", "Product A", 10, 200.0),
    ("2024-01-01", "South", "Product B", 5, 300.0),
    ("2024-01-02", "North", "Product A", 20, 400.0),
    ("2024-01-02", "South", "Product B", 10, 600.0),
    ("2024-01-03", "East",  "Product C", 15, 375.0),
]
columns = ["date", "region", "product", "quantity", "revenue"]
sales_df = spark.createDataFrame(sales_data, columns)
sales_df.show()

+----------+------+---------+--------+-------+
|      date|region|  product|quantity|revenue|
+----------+------+---------+--------+-------+
|2024-01-01| North|Product A|      10|  200.0|
|2024-01-01| South|Product B|       5|  300.0|
|2024-01-02| North|Product A|      20|  400.0|
|2024-01-02| South|Product B|      10|  600.0|
|2024-01-03|  East|Product C|      15|  375.0|
+----------+------+---------+--------+-------+



In [0]:
# Product Name and Total Revenue

from pyspark.sql.functions import sum
display(sales_df.groupBy("product").agg(sum("revenue").alias("total_revenue")))

product,total_revenue
Product A,600.0
Product B,900.0
Product C,375.0


In [0]:
# Total revenue by region

sales_df.groupBy("region").agg(sum("quantity").alias("total_quantity"),sum("revenue").alias("total_revenue")).show()

+------+--------------+-------------+
|region|total_quantity|total_revenue|
+------+--------------+-------------+
| North|            30|        600.0|
| South|            15|        900.0|
|  East|            15|        375.0|
+------+--------------+-------------+



In [0]:
# Average revenue per product

from pyspark.sql.functions import avg

sales_df.groupBy("product").agg(
    (sum("revenue")/sum("quantity")).alias("average_revenue")).show()

+---------+---------------+
|  product|average_revenue|
+---------+---------------+
|Product A|           20.0|
|Product B|           60.0|
|Product C|           25.0|
+---------+---------------+



In [0]:
# Find the top performing region (highest revenue)

from pyspark.sql.functions import sum

top_region = sales_df.groupBy("region").agg(
    sum("revenue").alias("total_revenue")
).orderBy("total_revenue", ascending=False)

top_region.show(1)  # Show only the top region

+------+-------------+
|region|total_revenue|
+------+-------------+
| South|        900.0|
+------+-------------+
only showing top 1 row

